# Homework - Grover MaxCut

The places where you have enter code are marked with `# YOUR CODE HERE`.

In [1]:
import cirq
from cirq import H, X, Y, Z, CX, CCZ, inverse

## Question 1 (4 points)

Write a function, `oracle010`, that implements an oracle that marks the state $|010 \rangle$. The function `oracle010` has

* input: `qq`, a 3-qubit register 
* returns: `None`

The function should append a sequence of gates to `qq` to mark the state $|010\rangle$ only. Don't append any measurements to `qq`.

To help you test the function, we have provided the `grover_diffusion` and `grover` functions.

In [2]:
def oracle010(qq):

    # Mark the state |010>
    yield X(qq[0])  
    yield X(qq[2])  
    
    # Apply a controlled-Z gate with control on all qubits
    yield H(qq[1])  # Apply Hadamard to the middle qubit
    yield CCZ(qq[0], qq[2], qq[1])  # Controlled-Controlled-Z gate
    yield H(qq[1])  # Apply Hadamard to the middle qubit again
    
    # Uncompute: flip the qubits back
    yield X(qq[0])  # Flip the first qubit back
    yield X(qq[2])  # Flip the third qubit back

In [3]:
# visualize your implemented gates
qqTest = cirq.LineQubit.range(3)
circuit = cirq.Circuit()
circuit.append(oracle010(qqTest))
circuit

0: ───X───@───X───
          │
1: ───H───@───H───
          │
2: ───X───@───X───

In [4]:
# To check your solution, we need some to implement grover
def grover_diffusion(qq,n):
    yield H.on_each(*qq)
    yield X.on_each(*qq)
    yield Z(qq[n-1]).controlled_by(*(qq[0:n-1]))
    yield X.on_each(*qq)
    yield H.on_each(*qq)

In [5]:
def grover(trials_number):
    n=3
    qq = cirq.LineQubit.range(n)
    circuit = cirq.Circuit()
    circuit.append(H.on_each(*qq))  

    for i in range(2):
        circuit.append(oracle010(qq))
        circuit.append(grover_diffusion(qq,n))
    circuit.append(cirq.measure(*qq, key='result'))

    # determine the statistics of the measurements
    s = cirq.Simulator() 
    samples = s.run(circuit, repetitions=trials_number)

    def bitstring(bits):
        return "".join(str(int(b)) for b in bits)

    counts = samples.histogram(key="result",fold_func=bitstring)
    print(counts)
    return counts.get('010')

In [6]:
# run grover to test if your function gives the right answer
grover(100)

Counter({'111': 16, '000': 14, '011': 14, '001': 13, '100': 13, '110': 11, '101': 10, '010': 9})


9

In [7]:
# hidden tests in this cell will be used for grading.

## Question 2 (6 points)

Graph $G$ has 5 vertices and 6 edges: (0,3), (0,4), (1,3), (1,4), (2,3), (2,4).

Write an oracle for the graph $G$ to check whether it admits a valid 2-coloring. 

The function `oracle2` has

* input: `qq`, a 12-qubit register 
* returns: `None`

The function should append only a sequence of gates to `qq`. It should not append any measurements to `qq`.

Use qubits 0-4 for the vertices, 5-10 for the edges and 11 as the ancilla. 

You can test the oracle with the provided `grover_diffusion`, `grover` and `oracle_computation2` functions.

In [8]:
qq = cirq.LineQubit.range(12)

In [9]:
def edge_check(a,b,c):
    yield CX(qq[a], qq[c])
    yield CX(qq[b], qq[c])

In [10]:
def oracle2(qq):
    yield edge_check(0,3,5)
    yield edge_check(0,4,6)
    yield edge_check(1,3,7)
    yield edge_check(1,4,8)
    yield edge_check(2,3,9)
    yield edge_check(2,4,10)
    

In [11]:
# We need some code so you can check your solution
def oracle_computation2(qq):
    yield oracle2(qq)
    yield Z(qq[11])
    yield inverse(oracle2(qq))  

In [12]:
def grover2(trials_number):    
    import cirq
    from cirq import X, H, Z, inverse, CX
    s = cirq.Simulator()

    qq = cirq.LineQubit.range(12)
    n=5
    
    circuit = cirq.Circuit()
    circuit.append(H.on_each(*(qq[0:n])))
    for i in range(2):
        circuit.append(oracle_computation2(qq))
        circuit.append(grover_diffusion(qq,n))

    circuit.append(cirq.measure(*(qq[0:n]), key='result'))

    # determine the statistics of the measurements
    samples = s.run(circuit, repetitions=trials_number)
    result = samples.measurements["result"]

    def bitstring(bits):
        return "".join(str(int(b)) for b in bits)

    counts = samples.histogram(key="result",fold_func=bitstring)
    return counts

In [19]:
#You can use this cell to test your solution
shots=1000
grover2(shots)

Counter({'01101': 41,
         '11101': 37,
         '11011': 37,
         '01011': 37,
         '01100': 37,
         '01010': 36,
         '01000': 36,
         '00010': 35,
         '01001': 35,
         '00001': 33,
         '11001': 33,
         '10110': 33,
         '00000': 32,
         '10011': 32,
         '10001': 32,
         '01111': 31,
         '00111': 31,
         '11010': 31,
         '10100': 31,
         '01110': 30,
         '11100': 30,
         '00100': 29,
         '00101': 29,
         '00011': 29,
         '10101': 29,
         '11110': 27,
         '10000': 26,
         '10111': 25,
         '10010': 25,
         '11000': 25,
         '00110': 24,
         '11111': 22})

In [15]:
# hidden tests in this cell will be used for grading.

## Question 3 (10 points)

Graph $G$ has 4 vertices and 5 edges: (0,1), (0,2), (0,3), (1,2), (1,3)

Write an oracle for the graph $G$ to check whether there exists a coloring with at least 4 edges connecting vertices with different colors.

The function `oracle3` has

* input: `qq`, a 13-qubit register 
* returns: `None`

The function should append only a sequence of gates to `qq`. It should not append any measurements to `qq`.

Use qubits 
- 0-3 for the vertices,
- 4-8 for the edges,
- 9-11 for the addition (remember we need three qubits here for addition unlike the last question), and
- 12 as the ancilla.

You can test the oracle with the provided `grover_diffusion`, `grover` and `oracle_computation3` functions.

In [20]:
qq = cirq.LineQubit.range(13)

In [21]:
def oracle3(qq):
    yield edge_check(0,1,4)
    yield edge_check(0,2,5)
    yield edge_check(0,3,6)
    yield edge_check(1,2,7)
    yield edge_check(1,3,8)

In [22]:
# We need some code so you can check your solution
def oracle_computation3(qq):
    yield oracle3(qq)
    yield Z(qq[12])
    yield inverse(oracle3(qq))  

In [23]:
import cirq
from cirq import X, H, Z, inverse, CX, CCX
    
def grover3(trials_number):    
    s = cirq.Simulator()

    qq = cirq.LineQubit.range(13)
    n=4

    circuit = cirq.Circuit()
    circuit.append(H.on_each(*(qq[0:n])))
    for i in range(2):
        circuit.append(oracle_computation3(qq))
        circuit.append(grover_diffusion(qq,n))

    circuit.append(cirq.measure(*(qq[0:n]), key='result'))

    # determine the statistics of the measurements
    samples = s.run(circuit, repetitions=trials_number)
    result = samples.measurements["result"]

    def bitstring(bits):
        return "".join(str(int(b)) for b in bits)

    counts = samples.histogram(key="result",fold_func=bitstring)
    return counts

In [27]:
#You can use this cell to test your solution
shots=1000
grover3(shots)

Counter({'0100': 73,
         '1101': 73,
         '1110': 72,
         '1010': 70,
         '0001': 69,
         '1111': 68,
         '1001': 67,
         '0011': 65,
         '1100': 60,
         '0010': 59,
         '0110': 59,
         '1000': 58,
         '0101': 56,
         '0000': 54,
         '1011': 50,
         '0111': 47})

In [28]:
# hidden tests in this cell will be used for grading.